<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/multi_material-lungman_phantom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
#
#  Copyright 2025 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Low-dose CT scan with the Lungman phantom

In this notebook you will use the knowledge learned in [Multi-material example: Lungman phantom](multi_material-CT_scan.ipynb). 
For the sake of realism, we will use an antrhopomorphic phantom this time, the [Lungman anthropomorphic chest phantom](https://doi.org/10.1117/1.JMI.5.1.013504) (Kyoto Kagaku, Tokyo, Japan). 
When it was originally scanned, there were nodules. 
It makes it a perfect example for illustrating low-dose CBCT in oncology. 
There are two ways to reduce the radiation dose in CT: lower the exposure, or reduce the number of projections taken. 
This notebook let you explore both strategies. 

![Change picture](change picture)

<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the required packages.
</div>

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install gvxr SimpleITK k3d spekpy xpecgen

## Aims of this session


1. Simulate a low-dose CBCT scan acquisition using gVXR, either
    - adding quite a bit of noise to mimic a low exposure, or
    - reducing the number of projections taken around the patient.
2. Reconstruct the CT volume using the Core Imaging Library (CIL).

![Change picture -- Screenshot of the 3D environment using K3D]()

## Summary of the simulation parameters

| Parameters | Values | Units | Function call |
|------------|--------|-------|---------------|
| Source to patient distance | ??? | mm | `gvxr.setSourcePosition(...)` |
| Source type (beam shape) | Point source (cone beam) | | `gvxr.usePointSource()` |
| Beam spectrum | Polychromatic: 100 | kV | `gvxr.setMonoChromatic(...)` |
| Source to detector distance | 100–182.5 | cm | `gvxr.setDetectorPosition(...)` |
| Detector model | Varian a-Si | |  |
| Detector orientation | [0, 0, -1] |  | `gvxr.setDetectorUpVector(...)` |
| Detector resolution | 1190 &times; 1190 | pixels | `gvxr.setDetectorNumberOfPixels(...)` |
| Pixel spacing | 0.33613445 &times; 0.33613445 | mm | `gvxr.setDetectorPixelSize(...)` |
| Scintillator material | GOS phosphor |  | ??? |
| Scintillator thickness | 0.290 | mm |   |
| Detector "cover" | 1 | mm |  |


kVp: 100
Exposure time (in msec): 285
Exposure time (in sec): 0.28500000000000003
X-ray Tube Current (in mA): 175
Exposure (in mAs): 83
Distance Source to Detector (in mm): 1085.6
Distance Source to Patient (in mm): 595.0
Rows and columns: 512 512
Pixel Spacing (in mm):  [0.625, 0.625]
Slice Thickness (in mm):  1.0
Volume size (in px):  512x512x426
Corrected slice Thickness (in mm):  -86.1


<!-- doi: 10.4172/2155-9619.1000354 -->
<!-- DOI 10.1088/1361-6560/ab12aa -->

<!-- a-Si 1200 -->
<!-- 1190 × 1190 -->
<!-- 0.34 0.33613445 -->
<!-- GOS phosphor: A layer of phosphor that is 0.290 mm thick -->
<!-- Copper build-up: A component that is the same thickness as the MLI (1 mm)  -->


<!-- Tube voltage: 40–150 kV -->
<!-- Source to image distance (SID): 100–182.5 cm -->
<!-- Slice thickness: 1.0–5.0 mm -->
<!-- Reconstructed volume resolution: Up to 512 × 512 -->
<!-- Scan modes: Half-fan (HF) and full-fan (FF) -->
<!-- Field of view (FOV): 24 cm for FF and 45 cm for HF -->
<!-- X-ray source peak voltage: 100 kVp for head scans and 125 kVp for abdominal scans  -->

## Import packages

- `os` to create the output directory if needed
- `matplotlib` to show 2D images
- `SimpleITK` to load the DICOM file
- `gvxr` to simulate X-ray images

In [ ]:
import math
import os # Create the output directory if necessary
import numpy as np # Who does not use Numpy?
import datetime

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table

font = {'family' : 'serif',
         'size'   : 15
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

import SimpleITK as sitk


import base64

import ipywidgets as widgets

from gvxrPython3 import gvxr
from gvxrPython3.utils import loadSpekpySpectrum
from gvxrPython3.utils import visualise
from gvxrPython3.utils import plotScreenshot
from gvxrPython3.utils import interactPlotPowerLaw # Plot the X-ray image using a Power law look-up table

# Use temporary bug fix
if os.path.exists("gvxr2json.py"):
    print("Use temporary bug fix")
    import gvxr2json
#Use the file provided by gVXR's package
else:
    print("Use the file provided by gVXR's package")
    from gvxrPython3 import gvxr2json

from gvxrPython3.gVXRDataReader import *

# from cil.utilities.jupyter import islicer
from cil.processors import TransmissionAbsorptionConverter
from cil.utilities.display import show_geometry, show2D
from cil.utilities.jupyter import islicer, link_islicer
from cil.recon import FBP, FDK
from cil.plugins.astra.processors.FDK_Flexible import FDK_Flexible

# GD_LS
from cil.optimisation.algorithms import GD
from cil.plugins.astra import ProjectionOperator
from cil.optimisation.functions import LeastSquares
from cil.processors import Slicer
from cil.plugins.ccpi_regularisation.functions import FGP_TV

# # SPDHG
from cil.optimisation.algorithms import SPDHG
from cil.optimisation.operators import BlockOperator
from cil.optimisation.functions import BlockFunction, L2NormSquared


from utils import downloadLungman, extractLungmanSTL, extractLungmanDX, loadLungmanMeshes

## Getting the data ready

Where to save the data.

In [4]:
output_path = "output_data/multi_material-CT_scan-low_dose"
if not os.path.exists(output_path):
    os.makedirs(output_path);

Download the data from [Zenodo](https://zenodo.org/records/10782644).

In [5]:
zip_fname, lungman_path, mesh_path = downloadLungman()

Extract the STL files from the ZIP file.

In [6]:
stl_fname_set = extractLungmanSTL(zip_fname, lungman_path)

Extract the Digital Radiograph (DX) (a DICOM file) from the ZIP file.

In [7]:
DX_DICOM_fname_set = extractLungmanDX(zip_fname, lungman_path)

## Read the ground truth Lungman data

<!-- The data is store in DICOM files. The first slice is loaded manually to extract the metadata. The volume is loaded as a DICOM series. -->


In [8]:
# file_reader = sitk.ImageFileReader()
# file_reader.SetFileName(DICOM_fname_set[0])
# file_reader.ReadImageInformation()
# file_reader.LoadPrivateTagsOn()
# temp_image0 = file_reader.Execute()

# file_reader = sitk.ImageFileReader()
# file_reader.SetFileName(DICOM_fname_set[1])
# file_reader.ReadImageInformation()
# file_reader.LoadPrivateTagsOn()
# temp_image1 = file_reader.Execute()
# first_slice_ref = sitk.GetArrayFromImage(temp_image1).astype(float)[0]

# file_reader.SetFileName(DICOM_fname_set[2])
# file_reader.ReadImageInformation()
# file_reader.LoadPrivateTagsOn()
# temp_image3 = file_reader.Execute()
# middle_slice_ref = sitk.GetArrayFromImage(temp_image3).astype(float)[0]

# file_reader.SetFileName(DICOM_fname_set[3])
# file_reader.ReadImageInformation()
# file_reader.LoadPrivateTagsOn()
# temp_image4 = file_reader.Execute()
# last_slice_ref = sitk.GetArrayFromImage(temp_image4).astype(float)[0]

Read the DICOM file.

In [9]:
reader = sitk.ImageFileReader();
reader.SetImageIO("GDCMImageIO");
reader.SetFileName(DX_DICOM_fname_set[0]);
reader.LoadPrivateTagsOn();
reader.ReadImageInformation();    
volume = reader.Execute();
raw_reference = sitk.GetArrayFromImage(volume)[0];

Extract information useful for the simulation from the DICOM file:

- The image size and the physical pixel spacing (i.e. not taking into account the magnification),
- The number of pixels,
- The distance from the source to the detector, and
- The distance from the source to the patient.

In [ ]:
# Extract the information from the DICOM header
imager_pixel_spacing = np.array(volume.GetMetaData("0018|1164").split("\\")).astype(np.single);
detector_element_spacing = np.array(volume.GetMetaData("0018|7022").split("\\")).astype(np.single);
print("Imager Pixel Spacing (in mm): ", imager_pixel_spacing, "(with magnification)");
print("Detector Element Spacing (in mm): ", detector_element_spacing, "(without magnification)");

# Extract the number of pixels
size = volume.GetSize()[0:2]
print("Image size (in pixels): ", str(size[0]) + " x " + str(size[1]))

# Extract the information from the DICOM header
distance_source_to_detector = float(volume.GetMetaData("0018|1110"))
distance_source_to_patient = float(volume.GetMetaData("0018|1111"))

print("Distance Source to Detector: ", distance_source_to_detector, "mm")
print("Distance Source to Patient: ", distance_source_to_patient, "mm")

We also extract the visualisation window to show the image using the 'harder' window.

In [ ]:
window_centre = int(volume.GetMetaData("0028|1050").split("\\")[1]) # Use 0 for normal, 1 for harder, 2 for softer
window_width = int(volume.GetMetaData("0028|1051").split("\\")[1]) # Use 0 for normal, 1 for harder, 2 for softer

print("Window Center used: ", window_centre)
print("Window Width used: ", window_width)

vmin = window_centre - window_width / 2
vmax = window_centre + window_width / 2

view_position = volume.GetMetaData("0018|5101")

In [ ]:
fig = plt.figure();
plt.imshow(raw_reference, cmap="gray", vmin=vmin, vmax=vmax,
                             extent=[0,(raw_reference.shape[1]-1)*imager_pixel_spacing[0],0,(raw_reference.shape[0]-1)*imager_pixel_spacing[1]])
plt.title("Digital radiograph of the Lungman phantom")
plt.xlabel("Pixel position\n(in mm)")
plt.ylabel("Pixel position\n(in mm)")
plt.colorbar()
plt.show()


## Extract experiment parameters from the DICOM metadata

"Slice thickness" is calculated from the positions of two neighbouring slices.

In [13]:
# # Extract the information from the DICOM header
# imager_pixel_spacing = np.array(volume.GetMetaData("0018|1164").split("\\")).astype(np.single);
# detector_element_spacing = np.array(volume.GetMetaData("0018|7022").split("\\")).astype(np.single);
# print("Imager Pixel Spacing (in mm): ", imager_pixel_spacing, "(with magnification)");
# print("Detector Element Spacing (in mm): ", detector_element_spacing, "(without magnification)");


In [ ]:
kvp = float(volume.GetMetaData("0018|0060"))
exposure_time_in_ms = int(volume.GetMetaData("0018|1150"))
exposure_time_in_sec = 0.001 * exposure_time_in_ms
xray_tube_current_in_mA = int(volume.GetMetaData("0018|1151"))
xray_tube_current_in_mA = int(volume.GetMetaData("0018|1153"))
exposure_in_mAs = int(volume.GetMetaData("0018|1152"))
exposure_in_uAs = int(volume.GetMetaData("0018|1153"))
distance_source_to_detector = float(volume.GetMetaData("0018|1110"))
distance_source_to_patient = float(volume.GetMetaData("0018|1111"))
detector_element_spacing = np.array(volume.GetMetaData("0018|7022").split("\\")).astype(np.single);

rows = int(volume.GetMetaData("0028|0010"))
columns = int(volume.GetMetaData("0028|0011"))

ref_size = [rows, columns, 426] #ref_volume.GetSize()

# slice_thickness = float(volume.GetMetaData("0018|0050"))

print("kVp:", kvp)
print("Exposure time (in msec):", exposure_time_in_ms)
print("Exposure time (in sec):", exposure_time_in_sec)
print("X-ray Tube Current (in mA):", xray_tube_current_in_mA)
print("Exposure (in mAs):", exposure_in_mAs)
print("Exposure (in uAs):", exposure_in_uAs)
print("Distance Source to Detector (in mm):", distance_source_to_detector)
print("Distance Source to Patient (in mm):", distance_source_to_patient)
print("Rows and columns:", rows, columns)
print("Detector Element Spacing (in mm): ", detector_element_spacing, "(without magnification)");
# print("Slice Thickness (in mm): ", slice_thickness)
# print("Volume size (in px): ", str(ref_size[0]) + "x" + str(ref_size[1]) + "x" + str(ref_size[2]))

# slice_thickness = 0.7 #ref_volume.GetSpacing()[2]
# slice_thickness = abs(float(temp_image0.GetMetaData("0020|1041")) - float(temp_image1.GetMetaData("0020|1041")))

# print("Corrected slice Thickness (in mm): ", slice_thickness)

# voxel_size = [detector_element_spacing[0], detector_element_spacing[0], detector_element_spacing[1]]

<!-- Calculate the diagonal to make the detector size big enough to fit the scan. -->

In [15]:
# diagonal = 1 + round(math.sqrt(math.pow(columns * pixel_spacing[0], 2) + math.pow(rows * pixel_spacing[1], 2)) / pixel_spacing[0])

# if diagonal % 2 == 0:
#     diagonal + 1

## 1. Create an OpenGL context

The first step is to create the simulation environment, known here as "OpenGL context".
`gvxr.createOpenGLContext` will try to find the most suitable environment possible regardless of the operating system. This is an alternative function to `gvxr.createNewContext` used in [test_installation.ipynb](test_installation.ipynb).

In [16]:
gvxr.createOpenGLContext();

# or 
# backend = "OPENGL";
# backend = "EGL";
# gvxr.createNewContext(backend);
#
# with backend a string. Two backends are currently available:
#
#     "OPENGL": makes use of the windowing ability of your system. It can be used for realtime visualisations. 
#               It is available on Windows, GNU/Linux and MacOS computers.
#     "EGL": is for offscreen rendering on GNU/Linux computers. That's the option for cloud instances and supercomputers.

We increase the size of the visualisation framebuffer to generate higher resolution screenshots. It does not affect the simulation.

In [17]:
gvxr.setWindowSize(1000, 1000)

## 2. Set the Sample

A sample is define by its geometry (surface) and material composition. Note that you can transform (translate, scale and rotate) a sample.

In [18]:
loadLungmanMeshes(mesh_path)
skin_bbox = gvxr.getNodeOnlyBoundingBox("skin", "mm")

## 3. Set the Detector

A detector is defined by its position, orientation, pixel resolution and the space between the centre of two consecutive pixels along its two axes. Here we also set a scintillator.

In [ ]:
gvxr.setDetectorPosition(0, 
    -(skin_bbox[1] + distance_source_to_patient - distance_source_to_detector), 
    0, 
    "mm");
gvxr.setDetectorUpVector(0, 0, 1);
gvxr.setDetectorNumberOfPixels(columns, rows);
gvxr.setDetectorPixelSize(*detector_element_spacing, "mm");
gvxr.setScintillator("CsI", 600, "um");

print("Detector position:", gvxr.getDetectorPosition("mm"), "mm")
print("Detector up vector:", gvxr.getDetectorUpVector())
print("Detector number of pixels:", gvxr.getDetectorNumberOfPixels())
print("Pixel spacing:", gvxr.getDetectorPixelSpacing("mm"), "mm")


Plot the energy response of the detector.

In [ ]:
detector_response = np.array(gvxr.getEnergyResponse("keV"));

plt.figure(figsize= (20,10))
# plt.title("Detector response")
plt.plot(detector_response[:,0], detector_response[:,1])
plt.xlabel('Incident energy: E (in keV)')
plt.ylabel('Detector energy response: $\\delta$(E) (in keV)')

plt.tight_layout()

plt.savefig(output_path + '/detector_response.pdf')
plt.savefig(output_path + '/detector_response.png')

## 4. Set the X-ray source

We must set it's position and beam shape. We will use the distance from the DICOM file.

In [ ]:
gvxr.setSourcePosition(0, 
    -(distance_source_to_detector + skin_bbox[1] + distance_source_to_patient - distance_source_to_detector), 
    0, 
    "mm");    
gvxr.useParallelSource();

print("Source position:", gvxr.getSourcePosition("mm"), "mm")
print("Source shape:", gvxr.getSourceShape())


## 5. Set the Spectrum

We define here the number of photons and their kinetic energy.
Again, we will use the 


In [ ]:
filtration = None;

source_detector_distance = np.linalg.norm(np.array(gvxr.getSourcePosition("cm")) - np.array(gvxr.getDetectorPosition("cm")))

spectrum = loadSpekpySpectrum(kvp, 
    filters=filtration,
    th_in_deg=12,
    mAs=exposure_in_mAs,
    z=source_detector_distance
    );


Plot the beam spectrum computed with Spekpy.

In [ ]:
plt.figure(figsize= (20,10))
# plt.title("Beam spectrum")
plt.bar(spectrum[1], spectrum[2], width=1)
plt.xlabel('Energy in keV')
plt.ylabel('Probability distribution of photons per keV')
plt.savefig(output_path + "/spectrum.pdf")
plt.show()

## 6. Compute the corresponding X-ray image.

It is possible to compute, retrieve and save an X-ray image as well as the path length of X-ray through an object.

In [24]:
x_ray_image = np.array(gvxr.computeXRayImage(), dtype=np.single) / gvxr.getTotalEnergyWithDetectorResponse();

In [ ]:
plotScreenshot()

In [ ]:
%matplotlib inline
from scipy import *
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from IPython.display import display
import spekpy as sp

number_of_white_images = 10
new_exposure_time_in_msec = None

def ms2mAs(ms):
    original_mAs = (exposure_in_uAs / 1000.0)
    original_current_in_mA = original_mAs / exposure_time_in_sec;
    return original_current_in_mA * (ms / 1000.0)

def interact_spectrum():
    ## Plot parameters
    xmin, xmax, nx = 0.0, 10.0, 50
    ymin, ymax = -1.2, 1.2
    pmin, pmax, pstep, pinit = -3.2, 3.2, 0.2, 0.0
    
    ## Set up the plot data
    x     = np.linspace(xmin, xmax, nx)
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(10,5))
    line, = axs[0].plot([], [], linewidth=2) # Initialize curve to empty data.

    ## Set up the figure axes, etc.
    axs[0].set_title("Tube spectrum")
    axs[0].set_xlim(xmin, xmax)
    # axs[0].set_ylim(ymin, ymax)
    axs[0].set_xlabel('Energy in keV')
    axs[0].set_ylabel('Number photons per keV per cm$^2$')
    
    axs[1].set_title("X-ray image\nUsing a log LUT")
    
    axs[2].set_title("Intensity profile\nmiddle row")
    # axs[2].set_xlim(xmin, xmax)
    # axs[2].set_ylim(ymin, ymax)
    axs[2].set_xlabel('Pixel position')
    axs[2].set_ylabel('Pixel intensity')
    line_profile, = axs[2].plot([], [], linewidth=2) # Initialize curve to empty data.

    plt.close()      # Don't show the figure yet.

    # Select the voltage
    kV_slider = widgets.IntSlider(
        value=100,
        min=20,
        max=200,
        step=1,
        description='kV:'
    )
    
    # Select the mAs
    exposure_time_slider = widgets.FloatSlider(
        value=exposure_time_in_ms,
        min=0.1,
        max=15,
        step=0.25,
        description='Exposure time in ms:',
    )
    
    white_slider = widgets.IntSlider(
        value=number_of_white_images,
        min=0,
        max=100,
        step=1,
        description='White images:',
    )
    
    ## Callback function
    def update_plot(kV, ms, white_images):
        
        global number_of_white_images, x_ray_image
        global new_exposure_time_in_msec

        number_of_white_images = white_images
        
        source_detector_distance = np.linalg.norm(np.array(gvxr.getSourcePosition("cm")) - np.array(gvxr.getDetectorPosition("cm")))

        # Generate and load the corresponding spectrum
        new_exposure_time_in_msec = ms
        spectrum = loadSpekpySpectrum(kvp, 
            filters=None,
            th_in_deg=12,
            mAs=ms2mAs(ms),
            z=source_detector_distance
            );
        
        if len(spectrum[1]) == len(spectrum[2]) and len(spectrum[1]) > 0:
            axs[0].set_xlim(0, kV_slider.value)
            axs[0].set_ylim(0, np.max(spectrum[2]))
            line.set_data(spectrum[1], spectrum[2])
        else:
            line.set_data([], [])
        
        gvxr.enablePoissonNoise()

        x_ray_image = np.array(gvxr.computeXRayImage()) / gvxr.getUnitOfEnergy("MeV")
        
        if white_images > 0:
            white_image = np.array(gvxr.getWhiteImage()) / gvxr.getUnitOfEnergy("MeV")
            for i in range(white_images - 1):
                white_image += np.array(gvxr.getWhiteImage()) / gvxr.getUnitOfEnergy("MeV")
            white_image /= white_images

            x_ray_image /= white_image

        x_ray_image[x_ray_image<0.0] = 1e-5
        axs[1].imshow(np.log(x_ray_image), cmap="gray")

        
        row = x_ray_image[x_ray_image.shape[0] // 2]
        line_profile.set_data(np.arange(len(row)), row)
        axs[2].set_xlim(0, len(row) - 1)
        axs[2].set_ylim(0, np.max(row))

        display(fig)
        
    ## Generate the user interface.
    interact(update_plot, kV=kV_slider, ms=exposure_time_slider, white_images=white_slider)

interact_spectrum();

In [ ]:
default_number_of_projections = gvxr.getOptimalNumberOfProjectionsCT()

projection_slider = widgets.IntSlider(
        value=default_number_of_projections,
        min=50,
        max=round(default_number_of_projections*1.5),
        step=1,
        description='Projections:',
    )



def on_button_clicked(_):
    projection_slider.value = default_number_of_projections
    
button = widgets.Button(description="Reset")
button.on_click(on_button_clicked)

display(widgets.VBox([projection_slider, button]))

In [ ]:
number_of_projections = projection_slider.value;

number_of_white_images = 10

translation_vector_in_mm = [
    skin_bbox[0] + (skin_bbox[3] - skin_bbox[0]) / 2.0,
    skin_bbox[1] + (skin_bbox[4] - skin_bbox[1]) / 2.0,
    skin_bbox[2] + (skin_bbox[5] - skin_bbox[2]) / 2.0,
]

start_time = datetime.datetime.now()

CT_output_path = os.path.join(output_path, "projections-" + str(number_of_projections)+"-" + str(new_exposure_time_in_msec) +"mAs")
gvxr.computeCTAcquisition(CT_output_path, # the path where the X-ray projections will be saved.
                                                                    # If the path is empty, the data will be stored in the main memory, but not saved on the disk.
                                                                    # If the path is provided, the data will be saved on the disk, and the main memory released.
                          os.path.join(output_path, "screenshots-" + str(number_of_projections)), # the path where the screenshots will be saved.
                                                                    # If kept empty, not screenshot will be saved.
                          projection_slider.value, # The total number of projections to simulate.
                          0, # The rotation angle corresponding to the first projection.
                          True, # A boolean flag to include or exclude the last angle. It is used to calculate the angular step between successive projections.
                          360,
                          number_of_white_images, # The number of white images used to perform the flat-field correction. If zero, then no correction will be performed.
                          *translation_vector_in_mm, # The location of the rotation centre.
                          "mm", # The corresponding unit of length.
                          *gvxr.getDetectorUpVector(), # The rotation axis
                          True # If true the energy fluence is returned, otherwise the number of photons is returned
                               # (default value: true)
);

end_time = datetime.datetime.now()
delta_time = end_time - start_time
total_run_time_in_sec = delta_time.total_seconds()
run_time_in_msec_per_frame = total_run_time_in_sec * 1000 / number_of_projections

print("Total runtime for", number_of_projections, "projections:", total_run_time_in_sec, "[sec]")
print("Runtime per frame:", run_time_in_msec_per_frame, "[msec]")

In [ ]:
# # Save the current simulation states in a JSON file.
# # It could be used to re-run the simulation, or to read the data with CIL.
# json_fname = os.path.join(output_path, "simulation-" + str(number_of_projections) + ".json");
# gvxr2json.saveJSON(json_fname);

In [ ]:
# Read the simulated data with CIL.
# reader = JSON2gVXRDataReader(json_fname);
reader = gVXRDataReader(CT_output_path, gvxr.getAngleSetCT());
data = reader.read()

In [ ]:
data_corr = TransmissionAbsorptionConverter(white_level=data.max())(data)

In [ ]:
ig = data_corr.geometry.get_ImageGeometry();

In [ ]:
data_corr.reorder(order='tigre')
FBP_reconstruction = FBP(data_corr, ig).run()

In [ ]:
show2D(FBP_reconstruction, cmap='gray', origin='upper-left')

In [ ]:
data_corr.reorder("astra")

ag = data_corr.geometry
ig = ag.get_ImageGeometry()

# ig.voxel_num_x = ref_size[0]
# ig.voxel_num_y = ref_size[1]
# ig.voxel_num_z = ref_size[2]

# ig.voxel_size_x = voxel_size[0]
# ig.voxel_size_y = voxel_size[1]
# ig.voxel_size_z = voxel_size[2]

A = ProjectionOperator(ig, ag, device="gpu")

In [ ]:
b = data_corr
f1 = LeastSquares(A, b)
x0 = ig.allocate(0.0)
f1(x0);

In [ ]:
GD_LS = GD(initial=x0, 
    objective_function=f1, 
    step_size=None, 
    max_iteration=1000, 
    update_objective_interval=10)

In [ ]:
GD_LS.run(300, verbose=1)

In [ ]:
# Define number of subsets
n_subsets = 10

# Initialize the lists containing the F_i's and A_i's
f_subsets = []
A_subsets = []

# Define F_i's and A_i's
for i in range(n_subsets):
    # Total number of angles
    n_angles = len(ag.angles)
    # Divide the data into subsets
    data_subset = Slicer(roi = {'angle' : (i,n_angles,n_subsets)})(data_corr)
    # Define F_i and put into list
    fi = 0.5*L2NormSquared(b = data_subset)
    f_subsets.append(fi)
    # Define A_i and put into list 
    ageom_subset = data_subset.geometry
    Ai = ProjectionOperator(ig, ageom_subset)
    A_subsets.append(Ai)

# Define F and K
F = BlockFunction(*f_subsets)
K = BlockOperator(*A_subsets)

# Define G (by default the positivity constraint is on)
alpha = 0.025
G = alpha * FGP_TV()

In [ ]:
# Setup and run SPDHG for 50 iterations
spdhg = SPDHG(f = F, g = G, operator = K,  max_iteration = 50,
            update_objective_interval = 10)
spdhg.run()

spdhg_recon = spdhg.solution    

In [ ]:
sl1 = islicer(FBP_reconstruction, title="FBP or FDK", cmap='gray', origin='upper-left')
sl2 = islicer(GD_LS.solution, title="Gradient Descent", cmap='gray', origin='upper-left')
sl3 = islicer(spdhg.solution, title="Stochastic Primal Dual Hybrid Gradient Algorithm", cmap='gray', origin='upper-left')
link_islicer(sl1, sl2, sl3)
# link_islicer(sl1, sl2)

# Cleaning up

Once we have finished it is good practice to clean up the OpenGL contexts and windows with the following command.

In [ ]:
gvxr.terminate();